In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [5]:
df=pd.read_csv('email.csv')
df.head()

,Email No.,sent,received,reply,important,spam,draft,sender,Prediction
0,Email 1,0,0,0,0,0,0,0,0
1,Email 2,0,0,0,0,0,0,0,0
2,Email 3,0,0,0,0,0,0,0,0
3,Email 4,0,0,0,0,0,0,0,0
4,Email 5,0,0,0,0,0,0,0,0


In [6]:
df.info()
df.isnull()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5172 entries, 0 to 5171
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Email No.   5172 non-null   object
 1   sent        5172 non-null   int64 
 2   received    5172 non-null   int64 
 3   reply       5172 non-null   int64 
 4   important   5172 non-null   int64 
 5   spam        5172 non-null   int64 
 6   draft       5172 non-null   int64 
 7   sender      5172 non-null   int64 
 8   Prediction  5172 non-null   int64 
dtypes: int64(8), object(1)
memory usage: 363.8+ KB


,Email No.,sent,received,reply,important,spam,draft,sender,Prediction
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
5167,False,False,False,False,False,False,False,False,False
5168,False,False,False,False,False,False,False,False,False
5169,False,False,False,False,False,False,False,False,False
5170,False,False,False,False,False,False,False,False,False


In [7]:
X=df.drop('Prediction', axis=1)

v= df.Prediction
y=np.array(v).reshape(-1,1)

In [8]:
df.drop('Email No.', axis=1, inplace=True)
X = df.drop('Prediction', axis=1)
y = df['Prediction'].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)


sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [9]:
input_dim = X_train.shape[1]

autoencoder = keras.Sequential([
    keras.layers.Input(shape=(input_dim,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(input_dim, activation='sigmoid')
])

In [10]:
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test))
encoder = keras.Model(inputs=autoencoder.input, outputs=autoencoder.layers[2].output)
for layer in encoder.layers:
    layer.trainable = False

Epoch 1/50
130/130 [==============================] - 2s 3ms/step - loss: 0.1729 - val_loss: 0.0787
Epoch 2/50
130/130 [==============================] - 0s 2ms/step - loss: 0.0693 - val_loss: 0.0617
Epoch 3/50
130/130 [==============================] - 0s 2ms/step - loss: 0.0560 - val_loss: 0.0573
Epoch 4/50
130/130 [==============================] - 0s 2ms/step - loss: 0.0530 - val_loss: 0.0564
Epoch 5/50
130/130 [==============================] - 0s 2ms/step - loss: 0.0521 - val_loss: 0.0560
Epoch 6/50
130/130 [==============================] - 0s 2ms/step - loss: 0.0515 - val_loss: 0.0559
Epoch 7/50
130/130 [==============================] - 0s 2ms/step - loss: 0.0512 - val_loss: 0.0557
Epoch 8/50
130/130 [==============================] - 0s 2ms/step - loss: 0.0511 - val_loss: 0.0555
Epoch 9/50
130/130 [==============================] - 0s 2ms/step - loss: 0.0510 - val_loss: 0.0555
Epoch 10/50
130/130 [==============================] - 0s 2ms/step - loss: 0.0510 - val_loss: 0.0554

In [11]:
classifier = keras.Sequential([
    encoder,
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

classifier.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
130/130 [==============================] - 1s 3ms/step - loss: 0.6239 - accuracy: 0.7157 - val_loss: 0.5962 - val_accuracy: 0.7169
Epoch 2/10
130/130 [==============================] - 0s 2ms/step - loss: 0.5889 - accuracy: 0.7230 - val_loss: 0.5863 - val_accuracy: 0.7285
Epoch 3/10
130/130 [==============================] - 0s 2ms/step - loss: 0.5954 - accuracy: 0.7240 - val_loss: 0.5843 - val_accuracy: 0.7246
Epoch 4/10
130/130 [==============================] - 0s 2ms/step - loss: 0.5901 - accuracy: 0.7242 - val_loss: 0.6021 - val_accuracy: 0.7130
Epoch 5/10
130/130 [==============================] - 0s 2ms/step - loss: 0.5825 - accuracy: 0.7237 - val_loss: 0.5897 - val_accuracy: 0.7140
Epoch 6/10
130/130 [==============================] - 0s 2ms/step - loss: 0.5859 - accuracy: 0.7244 - val_loss: 0.5841 - val_accuracy: 0.7246
Epoch 7/10
130/130 [==============================] - 0s 2ms/step - loss: 0.5837 - accuracy: 0.7244 - val_loss: 0.5811 - val_accuracy: 0.7246
Epoch 

In [12]:
y_pred = (classifier.predict(X_test) > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{confusion}')
print(f'Classification Report:\n{report}')

33/33 [==============================] - 0s 1ms/step
Accuracy: 0.7120772946859903
Confusion Matrix:
[[726   0]
 [298  11]]
Classification Report:
              precision    recall  f1-score   support

           0       0.71      1.00      0.83       726
           1       1.00      0.04      0.07       309

    accuracy                           0.71      1035
   macro avg       0.85      0.52      0.45      1035
weighted avg       0.80      0.71      0.60      1035

